<a href="https://colab.research.google.com/github/punyajoy/HateXplain/blob/master/Example_HateExplain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/punyajoy/HateXplain.git

Cloning into 'HateXplain'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 228 (delta 125), reused 98 (delta 40), pack-reused 0
Receiving objects: 100% (228/228), 4.43 MiB | 6.11 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
cd HateXplain/

/content/HateXplain


In [ ]:
!mkdir Saved/
!mkdir explanations_dicts/

In [ ]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip  -P Data/

--2020-12-20 18:12:51--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2020-12-20 18:12:51--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2020-12-20 18:12:51--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [ ]:
!unzip Data/glove.840B.300d.zip -d Data/

Archive:  Data/glove.840B.300d.zip
  inflating: Data/glove.840B.300d.txt  


In [ ]:
!rm Data/glove.840B.300d.zip

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 10.0MB 17.5MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 378kB 48.3MB/s 
     |████████████████████████████████| 81kB 12.6MB/s 
     |████████████████████████████████| 10.0MB 50.4MB/s 
     |████████████████████████████████| 501kB 56.7MB/s 
     |████████████████████████████████| 276kB 54.6MB/s 
     |████████████████████████████████| 17.3MB 197kB/s 
     |████████████████████████████████| 12.4MB 58.5MB/s 
     |████████████████████████████████| 24.2MB 90kB/s 
     |████████████████████████████████| 92kB 14.6MB/s 
     |████████████████████████████████| 676.9MB 25kB/s 
     |████████████████████████████████| 6.8MB 54.4MB/s 
     |████████████████████████████████| 2.1MB 45.4MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 184kB 60.3MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
     |████████████████████████████████| 890kB 43.6MB/

In [ ]:
!python convert_to_word2vec.py

tcmalloc: large alloc 2635227136 bytes == 0x33c8000 @  0x7f2fd191d001 0x7f2fce3b3de5 0x7f2fce418691 0x7f2fce41a76f 0x7f2fce4b30b8 0x50a4a5 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x50ad03 0x634e72 0x634f27 0x6386df 0x639281 0x4b0dc0 0x7f2fd1518bf7 0x5b259a


In [ ]:
from manual_training_inference import *

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt


Using TensorFlow backend.


In [ ]:
path_file='best_model_json/bestModel_birnnscrat.json'
with open(path_file,mode='r') as f:
    params = json.load(f)
for key in params:
    if params[key] == 'True':
          params[key]=True
    elif params[key] == 'False':
          params[key]=False
    if( key in ['batch_size','num_classes','hidden_size','supervised_layer_pos','num_supervised_heads','random_seed','max_length']):
        if(params[key]!='N/A'):
            params[key]=int(params[key])
        
    if((key == 'weights') and (params['auto_weights']==False)):
        params[key] = ast.literal_eval(params[key])

##### change in logging to output the results to neptune
params['logging']='local'
params['device']='cpu'
params['best_params']=False

if torch.cuda.is_available() and params['device']=='cuda':    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
else:
    print('Since you dont want to use GPU, using the CPU instead.')
    device = torch.device("cpu")
    
    
#### Few handy keys that you can directly change.
params['variance']=1
params['epochs']=5
params['to_save']=True
params['num_classes']=2
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']
if(params['num_classes']==2 and (params['auto_weights']==False)):
      params['weights']=[1.0,1.0]
        
#for att_lambda in [0.001,0.01,0.1,1,10,100]
train_model(params,device)

Since you dont want to use GPU, using the CPU instead.
[1.2301791 0.8423818]


0it [00:00, ?it/s]


======== Epoch 1 / 5 ========
Training...


481it [02:50,  2.81it/s]
2it [00:00, 13.20it/s]

avg_train_loss 295.3091210506066
model previously passed
Running eval on  train ...


...

61it [00:04, 13.99it/s]
2it [00:00, 13.87it/s]

 Accuracy: 0.70
 Fscore: 0.70
 Precision: 0.71
 Recall: 0.71
 Roc Auc: 0.00
 Test took: 0:00:04
model previously passed
Running eval on  test ...


61it [00:04, 14.09it/s]


 Accuracy: 0.71
 Fscore: 0.70
 Precision: 0.71
 Recall: 0.72
 Roc Auc: 0.00
 Test took: 0:00:04
best_val_fscore 0.7225254307538099
best_test_fscore 0.7331258744632605
best_val_rocauc 0
best_test_rocauc 0
best_val_precision 0.7231835522608054
best_test_precision 0.7326289095519865
best_val_recall 0.7310875851876457
best_test_recall 0.7403644165349076


1

In [ ]:
!python testing_with_rational.py birnn_scrat 100
!python testing_for_bias.py birnn_scrat 100
!python testing_with_lime.py birnn_scrat 100 100


2020-12-20 17:58:19.466517: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading english - 1grams ...
Using TensorFlow backend.
Since you dont want to use GPU, using the CPU instead.
total_data 20148
100% 20148/20148 [00:28<00:00, 695.58it/s]
attention_error: 0
no_majority: 919
tcmalloc: large alloc 2635227136 bytes == 0x400f4000 @  0x7fe117e3c1e7 0x7fe115945ca1 0x7fe1159af965 0x7fe1159b04fe 0x7fe115a499ce 0x50a4a

In [ ]:
!python testing_with_lime.py birnn_scrat 100 100

2020-12-20 18:04:12.241069: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading english - 1grams ...
Using TensorFlow backend.
total_data 1924
100% 1924/1924 [00:02<00:00, 705.02it/s]
Traceback (most recent call last):
  File "testing_with_lime.py", line 396, in <module>
    final_dict=get_final_dict_with_lime(params,model_to_use,test_data,topk=5)
  File "testing_with_lime.py", line 313, in get_final_dict_with_li

In [ ]:
!ls explanations_dicts

bestModel_birnnscrat_bias.json


Bias Calculation